In [138]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [139]:
dict_table = pickle.load(open("dict_tablas", "rb"))
df_fixture = pd.read_csv("df_2024_clean.csv")
df_historical_data = pd.read_csv("df_data_historica_clean.csv")

C:\Users\hitak\AppData\Local\Temp\ipykernel_16336\4132078071.py:1: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  dict_table = pickle.load(open("dict_tablas", "rb"))


In [87]:
df_home = df_historical_data[['Home', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['Away', 'HomeGoals', 'AwayGoals']]

df_home = df_home.rename(columns={'Home':'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'Away':'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Argentina,1.280000,1.280000
Bolivia,0.595238,1.857143
Brazil,1.791667,1.250000
Chile,0.954545,1.318182
Colombia,0.714286,1.095238
...,...,...
Peru,1.042553,1.574468
Qatar,1.000000,1.500000
United States,0.625000,2.125000


In [88]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

In [89]:
print(df_team_strength.index)


Index([' Argentina', ' Bolivia', ' Brazil', ' Chile', ' Colombia',
       ' Costa Rica', ' Ecuador', ' Haiti', ' Honduras', ' Jamaica', ' Japan',
       ' Mexico', ' Panama', ' Paraguay', ' Peru', ' Qatar', ' United States',
       ' Uruguay', ' Venezuela', 'Argentina ', 'Argentina ', 'Bolivia ',
       'Bolivia ', 'Brazil ', 'Brazil ', 'Chile ', 'Chile ', 'Colombia ',
       'Colombia ', 'Costa Rica ', 'Costa Rica ', 'Ecuador ', 'Ecuador ',
       'Haiti ', 'Haiti ', 'Honduras ', 'Honduras ', 'Jamaica ', 'Jamaica ',
       'Japan ', 'Japan ', 'Mexico ', 'Mexico ', 'Panama ', 'Panama ',
       'Paraguay ', 'Paraguay ', 'Peru ', 'Peru ', 'Qatar ', 'Qatar ',
       'United States ', 'United States ', 'Uruguay ', 'Uruguay ',
       'Venezuela ', 'Venezuela ', ' Argentina', ' Bolivia', ' Brazil',
       ' Chile', ' Colombia', ' Costa Rica', ' Ecuador', ' Haiti', ' Honduras',
       ' Jamaica', ' Japan', ' Mexico', ' Panama', ' Paraguay', ' Peru',
       ' Qatar', ' United States', ' Urugua

In [90]:
points_home, points_away = predict_points(' Venezuela', ' Colombia')
print((float(points_home), float(points_away)))

(0.5365175092651457, 2.271008404956709)


In [135]:
df_fixture_group_all = df_fixture[:32].copy()
df_fixture_group_23 = df_fixture[:24].copy()
df_fixture_knockout = df_fixture[24:28].copy()
df_fixture_Final = df_fixture[28:30].copy()
df_fixture_loser_Match = df_fixture[30:31].copy()


In [136]:
df_fixture_group_all

,Home,Away,Score,year
0,Argentina,Canada,Match 1,2024
1,Peru,Chile,Match 2,2024
2,Peru,Canada,Match 10,2024
3,Chile,Argentina,Match 9,2024
4,Argentina,Peru,Match 17,2024
5,Canada,Chile,Match 18,2024
6,Ecuador,Venezuela,Match 4,2024
7,Mexico,Jamaica,Match 3,2024
8,Ecuador,Jamaica,Match 12,2024
9,Venezuela,Mexico,Match 11,2024


In [145]:
dict_table["Group A"]


,Team,Pts
0,Mexico,0
1,Ecuador,0
2,Venezuela,0
3,Jamaica,0


In [148]:
for group in dict_table:
    print(f"Processing {group}")
    teams_in_group = dict_table[group]['Team'].values

SyntaxError: invalid syntax (3264834941.py, line 6)